In [ ]:
!pip install transformers langchain langchain-community chromadb sentence-transformers pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.5 MB/s eta 0:00:0

In [ ]:
import chromadb
import langchain
import transformers
from sentence_transformers import SentenceTransformer

In [ ]:
!pip install pymupdf wikipedia-api beautifulsoup4 requests

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 53.9 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=c8220808e71d5603d8cf5dea4eb02ed8c8be97496331f2d5b4e35e512127b8e6
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api


In [ ]:
import wikipediaapi

def get_wikipedia_content(title):
    wiki = wikipediaapi.Wikipedia(
        language="en",
        user_agent="AutomotiveResearchBot/1.0"
    )
    page = wiki.page(title)

    if page.exists():
        return page.text
    else:
        return "Page not found."

wiki_topics = [
    "Autonomous_car",
    "Automotive_engineering",
    "Electric_vehicle"
]

wikipedia_data = {topic: get_wikipedia_content(topic) for topic in wiki_topics}

with open("wikipedia_data.txt", "w", encoding="utf-8") as f:
    for topic, content in wikipedia_data.items():
        f.write(f"### {topic.replace('_', ' ')} ###\n\n{content}\n\n{'='*80}\n\n")

print("Wikipedia data collected and saved successfully.")


Wikipedia data collected and saved successfully.


In [ ]:
import requests
from bs4 import BeautifulSoup
import time

blog_sources = {
    "Waymo": "https://waymo.com/blog/",
    "Ars Technica": "https://arstechnica.com/cars/",
}

def get_full_article(link):
    try:
        response = requests.get(link, headers={"User-Agent": "Mozilla/5.0"})
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            paragraphs = soup.find_all("p")
            content = "\n".join([p.get_text() for p in paragraphs])
            return content
    except:
        return ""
    return ""

def scrape_blogs(url, source_name):
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if response.status_code != 200:
        return f"Failed to retrieve content from {source_name}"

    soup = BeautifulSoup(response.text, "html.parser")
    if source_name == "Waymo":
        articles = soup.find_all("a", class_="MuiTypography-root")[:5]
    elif source_name == "Ars Technica":
        articles = soup.find_all("h2")[:5]
    elif source_name == "InsideEVs":
        articles = soup.find_all("h3")[:5]

    blog_data = []
    for article in articles:
        link = article.find("a")
        if link and link.get("href"):
            article_url = link["href"]
            if not article_url.startswith("http"):
                article_url = url + article_url  # Handle relative links
            article_text = get_full_article(article_url)
            if article_text:
                blog_data.append(f"### Article: {article.text} ###\n\n{article_text}\n\n")

            time.sleep(2)

    return "\n".join(blog_data)

blog_content = {source: scrape_blogs(url, source) for source, url in blog_sources.items()}

with open("blog_data.txt", "w", encoding="utf-8") as f:
    for source, content in blog_content.items():
        f.write(f"### {source} Blogs ###\n\n{content}\n\n{'='*80}\n\n")

print("blog articles collected and saved successfully.")


blog articles collected and saved successfully.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving PDF3.pdf to PDF3.pdf
Saving PDF2.pdf to PDF2.pdf
Saving PDF1.pdf to PDF1.pdf


In [ ]:
import fitz  # PyMuPDF

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text("text") + "\n"
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
    return text

# Extract text from each uploaded PDF
pdf_files = ["PDF1.pdf", "PDF2.pdf", "PDF3.pdf"]
pdf_texts = {}

for pdf in pdf_files:
    pdf_texts[pdf] = extract_text_from_pdf(pdf)
    print(f"Extracted text from {pdf} (First 500 chars):\n", pdf_texts[pdf][:500])  # Print sample text

# Save extracted text to a file for later use
for pdf, text in pdf_texts.items():
    with open(f"{pdf}.txt", "w", encoding="utf-8") as f:
        f.write(text)

print("PDF text extraction complete. Text files saved.")


Extracted text from PDF1.pdf (First 500 chars):
 Automotive ADAS Systems
Overall Automotive ADAS System

Table of Contents
• ADAS overview
• ADAS Vehicle Architectures
• ADAS Technologies/Sensors
• Vision(Cameras) System
• LiDAR System
• Radar System
• GNSS/IMU System
• V2X System
• Sensor Fusion Example
2
ST Confidential

Automotive ADAS Systems
ADAS Overview

Overview of ADAS Technologies
4
Power 
Supply/Management
Targets 

ADAS Sensors - Needed for Perception
5
LIDAR
Radar
Cameras
GNSS antenna
Central Computer
Ultra-Sonics
Wheel Odometry

Extracted text from PDF2.pdf (First 500 chars):
 www.ti.com/adas	
2015
Advanced Driver 
Assistance (ADAS) 
Solutions Guide

2  | Advanced Driver Assistance Systems (ADAS) Guide 2015	
Texas Instruments
Advanced driver assistance (ADAS)
Table of contents
Texas Instruments commitment 
to automotive safety 
Advanced driver assistance
systems (ADAS) are one of the 
fastest-growing application areas in 
vehicles today as ADAS features are 
incorporated

In [ ]:
import chromadb

# Initialize ChromaDB in-memory
chroma_client = chromadb.PersistentClient(path="./chroma_db")  # Saves data persistently

# Create a collection
collection = chroma_client.get_or_create_collection(name="automotive_knowledge")

# Function to add data to ChromaDB
def add_to_chroma(text, source):
    collection.add(
        documents=[text],
        metadatas=[{"source": source}],
        ids=[source]
    )

# Load extracted text files
data_sources = {
    "wiki": "wikipedia_data.txt",
    "blogs": "blog_data.txt",
    "pdf1": "PDF1.pdf.txt",
    "pdf2": "PDF2.pdf.txt",
    "pdf3": "PDF3.pdf.txt"
}

# Store extracted data in ChromaDB
for source, file_path in data_sources.items():
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
        add_to_chroma(text, source)
        print(f"Added {source} to ChromaDB.")

print("All extracted text stored in ChromaDB successfully.")

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 43.0MiB/s]


Added wiki to ChromaDB.
Added blogs to ChromaDB.
Added pdf1 to ChromaDB.
Added pdf2 to ChromaDB.
Added pdf3 to ChromaDB.
All extracted text stored in ChromaDB successfully.


In [ ]:
# Retrieve stored documents from ChromaDB
results = collection.get()

print(f"Total Documents Stored: {len(results['ids'])}")
print("Sample Retrieved Document:", results['documents'][0][:500])  # Print first 500 chars

Total Documents Stored: 5
Sample Retrieved Document: ### Autonomous car ###

A self-driving car, also known as a  autonomous car (AC), driverless car, robotaxi, robotic car or robo-car, is a car that is capable of operating with reduced or no human input. Self-driving cars are responsible for all driving activities, such as perceiving the environment, monitoring important systems, and controlling the vehicle, which includes navigating from origin to destination.
As of late 2024, no system has achieved full autonomy (SAE Level 5). In December 2020,


In [ ]:
!nvidia-smi

Thu Apr  3 11:08:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from chromadb import PersistentClient

# Path to stored ChromaDB
chroma_db_path = "/content/chroma_db"

# Load ChromaDB
chroma_client = PersistentClient(path=chroma_db_path)
collection = chroma_client.get_or_create_collection(name="automotive_knowledge")

print("ChromaDB loaded successfully!")

ChromaDB loaded successfully!
